In [ ]:
# !pip install psycopg2-binary

In [106]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import os
import pandas as pd
import shutil

In [2]:
DB_PARAMS = {
    "user": "postgres",
    "password": "123456",
    "host": "127.0.0.1",
    "port": "5432",
    "database": "postgres"  
}

DB_NAME = "lego_database"

In [9]:
def database_exists(conn, db_name):
    """
    Verifica si una base de datos ya existe.
    """
    with conn.cursor() as cur:
        cur.execute("SELECT 1 FROM pg_database WHERE datname=%s", (db_name,))
        return cur.fetchone() is not None

In [3]:
def execute_sql_from_file(conn, file_path, **kwargs):
    """
    Executes SQL commands from a given file.
    """
    with open(file_path, 'r') as file:
        sql = file.read().format(**kwargs)
    with conn.cursor() as cur:
        cur.execute(sql)

In [33]:
def create_database(connection_params, db_name):
    try:
        # Conectar a la base de datos predeterminada para realizar la verificación y creación
        conn = psycopg2.connect(**connection_params)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        with conn.cursor() as cur:
            # Verificar si la base de datos ya existe
            cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (db_name,))
            exists = cur.fetchone()
            if exists:
                print(f"Database '{db_name}' already exists.")
            else:
                # Ejecutar el comando CREATE DATABASE si no existe
                cur.execute(f"CREATE DATABASE {db_name}")
                DB_PARAMS["database"] = db_name
                print(f"Database '{db_name}' created successfully.")
        conn.close()

    except Exception as e:
        print(f"Error creating database: {e}")

In [64]:
def create_tables(db_params):
    """
    Creates tables in the database.
    """
    
    sql_files = [
    # 'create_colors_table.sql',
    # 'create_part_categories_table.sql',
    # 'create_parts_table.sql',
    'create_themes_table.sql'
    # 'create_sets_table.sql',
    # 'create_inventories_table.sql',
    # 'create_inventory_parts_table.sql',
    # 'create_inventory_sets_table.sql'
]

    try:
        with psycopg2.connect(**db_params) as conn:
            conn.set_session(autocommit=True)
            
            for sql_file in sql_files:
                execute_sql_from_file(conn, f'sql/{sql_file}')
                print(f"Table from '{sql_file}' created.")
    except Exception as e:
        print(f"Error creating tables: {e}")

In [43]:
create_database(DB_PARAMS, DB_NAME)

Database 'lego_database' already exists.


In [65]:
create_tables(DB_PARAMS)

Table from 'create_themes_table.sql' created.


In [63]:
# import csv
# import sqlite3

# # Conexión a la base de datos (ajustar según sea necesario)
# conn = sqlite3.connect('lego_database.db')
# cursor = conn.cursor()

# # Función modificada para insertar datos desde un archivo CSV a una tabla
# def insert_data_from_csv(file_path, table_name):
#     with open(file_path, newline='', encoding='utf-8') as csvfile:
#         reader = csv.reader(csvfile)
#         columns = next(reader)  # Obtiene el encabezado para las columnas
#         placeholders = ', '.join(['?'] * len(columns))  # Crea marcadores de posición
#         columns_formatted = ', '.join(columns)  # Formatea las columnas para la consulta SQL
#         sql = f'INSERT INTO {table_name} ({columns_formatted}) VALUES ({placeholders})'
#         for row in reader:
#             print(row)
#             print(sql)
#             cursor.execute(sql, row)
#     conn.commit()

In [97]:
import csv
from psycopg2 import sql, connect, extensions

def insert_data_from_csv_to_db(db_params, file_path, table_name):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        columns = next(reader)  # Obtiene el encabezado para las columnas
        records = [[None if item == '' else item for item in row] for row in reader]

        # Inicia una conexión a la base de datos
        conn = connect(**db_params)
        conn.set_isolation_level(extensions.ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = conn.cursor()

        # Prepara la consulta SQL para insertar todos los registros
        placeholders = ', '.join(['%s'] * len(columns))  # Marcadores de posición para una fila
        all_placeholders = ', '.join([f"({placeholders})"] * len(records))  # Repite los marcadores para todas las filas
        query = sql.SQL("INSERT INTO {table} ({fields}) VALUES {values} ON CONFLICT DO NOTHING").format(
            table=sql.Identifier(table_name),
            fields=sql.SQL(', ').join(map(sql.Identifier, columns)),
            values=sql.SQL(all_placeholders)
        )

        # Aplana la lista de registros para pasarla como argumentos a la consulta
        flat_records = [item for sublist in records for item in sublist]

        try:
            cursor.execute(query, flat_records)
            print("Todos los registros insertados exitosamente.")
        except psycopg2.IntegrityError as e:
            if "violates foreign key constraint" in str(e):
                missing_part_num = str(e).split("Key (part_num)=(")[1].split(")")[0]
                print(f"Error: part_num {missing_part_num} no está presente en la tabla 'parts'.")
            else:
                print(f"Error al ejecutar la consulta: {e}")
        except (Exception, psycopg2.DatabaseError) as error:
            print(f"Error al ejecutar la consulta: {error}")
        finally:
            cursor.close()
            conn.close()
            print("Conexión cerrada.")

colors: No depende de otras tablas.  
part_categories: No depende de otras tablas.  
themes: Aunque tiene una relación jerárquica consigo misma, podemos insertar primero los temas sin padres (o insertar todos los temas y luego actualizar aquellos con parent_id).  
parts: Depende de part_categories.    
sets: Depende de themes.  
inventories: Depende de sets para el campo set_num.  
inventory_parts: Depende de inventories, parts, y colors.  
inventory_sets: Depende de inventories y sets

In [57]:
table = 'colors'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [58]:
table = 'part_categories'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [71]:
table = 'themes'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [72]:
table = 'parts'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [73]:
table = 'sets'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [74]:
table = 'inventories'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [104]:
query = "SELECT DISTINCT part_num FROM parts"
unique_part_nums =  execute_query(DB_PARAMS, query)
unique_part_nums = [part_num[0] for part_num in unique_part_nums]
unique_part_nums 

Conexión cerrada.


['`3069bpr0180',
 '0687b1',
 '0901',
 '0902',
 '0903',
 '0904',
 '1',
 '10',
 '10016414',
 '10019stk01',
 '10026stk01',
 '10029stk01',
 '10036stk01',
 '10039',
 '10048',
 '10049',
 '10049pr0001',
 '10050',
 '10051',
 '10051pr01',
 '10052',
 '10053',
 '10054',
 '10054pr0001',
 '10054pr0002',
 '10055',
 '10055pr0001',
 '10056',
 '10056pr0001',
 '10057',
 '10057pr0001',
 '10057pr0002',
 '10058',
 '10061',
 '10062',
 '10063',
 '10064',
 '10065',
 '10066',
 '10066pr0001',
 '10066pr0002',
 '10066pr0003',
 '10075cdb01',
 '10111',
 '10111apr0006',
 '10113',
 '10113pr0001',
 '10113pr0002',
 '10119',
 '10124',
 '10124pr0001',
 '10126',
 '10127',
 '10127stk01',
 '10127stk02',
 '10127stk03',
 '10128',
 '10128pr0001',
 '10128pr0002',
 '10129stk01',
 '10134stk01',
 '10144stk01',
 '10154',
 '10154pr0001',
 '10159stk01',
 '10164',
 '10164pr0001',
 '10165c01',
 '10166',
 '10166pr0001',
 '10166pr0003',
 '10168pb01',
 '10169',
 '10170',
 '10172',
 '10173',
 '10177',
 '10178',
 '10178pr0001a',
 '10178pr00

In [107]:
# df_inventory_parts = pd.read_csv('./raw/inventory_parts.csv')
# df_filtered = df_inventory_parts[~df_inventory_parts['part_num'].isin(unique_part_nums)]

# print(df_filtered)


# Paso 1: Crear una copia del archivo original con sufijo _OLDER
shutil.copyfile('./raw/inventory_parts.csv', './raw/inventory_parts_OLDER.csv')

# Paso 2: Leer el archivo original en un DataFrame
df_inventory_parts = pd.read_csv('./raw/inventory_parts.csv')

# Suponiendo que unique_part_nums ya está definido con los part_num existentes en parts
# df_filtered ya contiene los registros a eliminar, por lo que se invierte el filtro para mantener los deseados
df_to_keep = df_inventory_parts[df_inventory_parts['part_num'].isin(unique_part_nums)]

# Paso 3: Guardar el DataFrame modificado (sin los registros no deseados) en el archivo original
df_to_keep.to_csv('./raw/inventory_parts.csv', index=False)

print("Archivo actualizado y copia _OLDER creada.")

Archivo actualizado y copia _OLDER creada.


In [108]:
table = 'inventory_parts'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


In [109]:
table = 'inventory_sets'
insert_data_from_csv_to_db(DB_PARAMS, f'./raw/{table}.csv', table)

Todos los registros insertados exitosamente.
Conexión cerrada.


# Consultas.

In [36]:
def execute_query(db_params, query):
    try:
        conn = psycopg2.connect(**db_params)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = conn.cursor()
        
        cursor.execute(sql.SQL(query))
        
        if query.strip().lower().startswith("select"):
            records = cursor.fetchall()
            return records
        else:
            print("Consulta ejecutada exitosamente.")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error al ejecutar la consulta: {error}")
    finally:
        if conn is not None:
            cursor.close()
            conn.close()
            print("Conexión cerrada.")

In [67]:
query = "delete from themes"
execute_query(DB_PARAMS, query)

Consulta ejecutada exitosamente.
Conexión cerrada.


In [56]:
query = "select * from colors"
execute_query(DB_PARAMS, query)

Conexión cerrada.


[]

In [50]:
query = "INSERT INTO colors (id, name, rgb, is_trans) VALUES ('-1', 'Unknown', '0033B2', 'f')"

execute_query(DB_PARAMS, query)

Consulta ejecutada exitosamente.
Conexión cerrada.


# Observaciones.
En themes tuve que cambiar a None donde habia cadenas vacias para columans tipo int. Sino, no funcionaba el insert.  

para inventory_parts se agrego ON CONFLICT (pk_column) DO UPDATE SET para que el registro se actualice cuando encuentre conflicto por pk duplicada. Tambien se implemento un proceso que traia los id de la tabla parts y comparaba entre sus regsitros cuales apuntaban a un id inexistente en parts. Se creo una copia de seguridad del archivo y estos registrso fueron eliminados  previo a la insersion.

Otras tablas no hubo problemas.